In [1]:
import pandas as pd

import Clean_data as cld
import Transform_Data as tfd
import Visualize_Data as vld

# Create Master Lookup Table

Because of all the information that different datasets have to offer, I'm going to grab info from several tables to help ease the usability of the entire range of datasets.

### City-Level

In [2]:
df = pd.read_csv('Datasets/House Price/City_Zhvi_AllHomes.csv')

In [3]:
lookup_df = df[['RegionName', 'StateName',
       'CountyName']]

In [4]:
lookup_df.head(3)

,RegionName,StateName,CountyName
0,New York,NY,Queens County
1,Los Angeles,CA,Los Angeles County
2,Houston,TX,Harris County


In [5]:
lookup_df = lookup_df.rename(columns = {'RegionName': 'City', 'StateName': 'StateAbbreviation', 'CountyName':'County'})

In [6]:
lookup_df.head(3)

,City,StateAbbreviation,County
0,New York,NY,Queens County
1,Los Angeles,CA,Los Angeles County
2,Houston,TX,Harris County


In [7]:
left ='StateAbbreviation'
lookup_df = tfd.add_state_abbrev(lookup_df, left)


In [8]:
lookup_df.head(3)

,City,StateAbbreviation,County,State
0,New York,NY,Queens County,New York
1,Town of Islip,NY,Suffolk County,New York
2,Buffalo,NY,Erie County,New York


In [9]:

left ='State'
lookup_df = tfd.add_state_region(lookup_df, left)


            City StateAbbreviation              County     State
0       New York                NY       Queens County  New York
1  Town of Islip                NY      Suffolk County  New York
2        Buffalo                NY         Erie County  New York
3        Yonkers                NY  Westchester County  New York
4       Syracuse                NY     Onondaga County  New York


In [10]:
lookup_df.head(3)

,City,StateAbbreviation,County,State,Region
0,New York,NY,Queens County,New York,Northeastern
1,Town of Islip,NY,Suffolk County,New York,Northeastern
2,Buffalo,NY,Erie County,New York,Northeastern


In [11]:
lookup_df

,City,StateAbbreviation,County,State,Region
0,New York,NY,Queens County,New York,Northeastern
1,Town of Islip,NY,Suffolk County,New York,Northeastern
2,Buffalo,NY,Erie County,New York,Northeastern
3,Yonkers,NY,Westchester County,New York,Northeastern
4,Syracuse,NY,Onondaga County,New York,Northeastern
...,...,...,...,...,...
27324,Brunswick,VT,Essex County,Vermont,Northeastern
27325,North Westminster,VT,Windham County,Vermont,Northeastern
27326,West Burke,VT,Caledonia County,Vermont,Northeastern
27327,Jeffersonville,VT,Lamoille County,Vermont,Northeastern


In [12]:

lookup_df.to_csv('City_Lookup.csv', index=False)

#### State-Level

In [13]:
state_df = lookup_df.copy()
state_df = state_df[['StateAbbreviation', 'State', 'Region']]

In [14]:
state_df = state_df.drop_duplicates(subset=['State'], keep='first')

In [15]:
state_df.head(5)

,StateAbbreviation,State,Region
0,NY,New York,Northeastern
1216,CA,California,California
2328,TX,Texas,Southern
3846,IL,Illinois,Midwestern
5098,PA,Pennsylvania,Northeastern


In [16]:
state_df.to_csv('State_Lookup.csv', index=False)

### Add Numerical Data

Now that I've got 2 basic ways to oreganize data (and maybe a third if I add larger national regions) I'm going to append relevant data.


#### City-level


Because of some of the extreme complexity with my City data across all the different datasets, I'm restricting the scope of my findings to State and Regional.

#### State-level

I will first grab the most updated info of each type I can and append it to each state

In [17]:
state_lookup = pd.read_csv('State_Lookup.csv')

In [18]:
state_demographics = pd.read_csv('Datasets/Discoveries/State_real_estate_state_grouped.csv')

In [19]:
state_demographics.head(5)

,Unnamed: 0,state,debt,rent_mean,rent_median,family_mean,family_median,hc_mortgage_mean,hc_mortgage_median,hc_mean,...,rent_gt_35,rent_gt_40,rent_gt_50,home_equity,male_percent,female_percent,rent_mean_to_median,family_mean_to_median,hc_mortgage_mean_to_median,hc_mean_to_median
0,0,Alabama,0.546932,762.827471,727.570016,65183.236790,54682.685338,1193.630170,1096.835255,384.365807,...,0.421118,0.353131,0.261761,0.070659,0.487282,0.512718,35.257455,10500.551453,96.794915,28.874868
1,1,Alaska,0.603078,1173.006814,1124.563107,92408.808068,82148.174757,1797.745110,1734.572816,577.574834,...,0.321569,0.259487,0.178842,0.053943,0.522756,0.477244,48.443708,10260.633310,63.172294,22.681630
2,2,Arizona,0.644890,1087.049918,1038.711514,73134.038870,63459.156533,1410.504653,1328.961190,428.258502,...,0.396900,0.329222,0.236027,0.092913,0.496905,0.503095,48.338404,9674.882337,81.543463,25.330947
3,3,Arkansas,0.548378,713.879645,676.640449,63939.536360,53052.250000,1108.327466,1003.679775,366.514645,...,0.383911,0.320823,0.231195,0.039670,0.491218,0.508782,37.239195,10887.286360,104.647691,27.528690
4,4,California,0.720963,1472.168818,1421.218025,88315.668424,78506.768148,2212.152681,2164.707654,558.631314,...,0.466828,0.393292,0.289799,0.132255,0.494450,0.505550,50.950794,9808.900276,47.445026,35.427858


In [20]:
state_lookup = state_lookup.merge(state_demographics, how='outer',left_on='State', right_on='state')

In [21]:
state_lookup.loc[50, 'StateAbbreviation'] = 'DC'
state_lookup.loc[50, 'State'] = 'District of Columbia'
state_lookup.loc[50, 'Region'] = 'Special'
state_lookup.loc[51, 'StateAbbreviation'] = 'PR'
state_lookup.loc[51, 'State'] = 'Puerto Rico'
state_lookup.loc[51, 'Region'] = 'Special'


In [22]:
state_lookup.head(5)

,StateAbbreviation,State,Region,Unnamed: 0,state,debt,rent_mean,rent_median,family_mean,family_median,...,rent_gt_35,rent_gt_40,rent_gt_50,home_equity,male_percent,female_percent,rent_mean_to_median,family_mean_to_median,hc_mortgage_mean_to_median,hc_mean_to_median
0,NY,New York,Northeastern,32,New York,0.625454,1266.696603,1223.602505,88530.777090,78968.387474,...,0.445111,0.381864,0.293017,0.102228,0.484546,0.515454,43.094098,9562.389616,46.073528,14.840002
1,CA,California,California,4,California,0.720963,1472.168818,1421.218025,88315.668424,78506.768148,...,0.466828,0.393292,0.289799,0.132255,0.494450,0.505550,50.950794,9808.900276,47.445026,35.427858
2,TX,Texas,Southern,44,Texas,0.553786,990.034921,944.247280,76901.574345,66839.803002,...,0.381331,0.315971,0.226842,0.036889,0.495208,0.504792,45.787641,10061.771343,77.061139,28.079996
3,IL,Illinois,Midwestern,13,Illinois,0.655639,1027.805758,976.730745,82451.122825,72965.984723,...,0.403694,0.341537,0.256194,0.108531,0.491094,0.508906,51.075014,9485.138101,82.864832,21.377380
4,PA,Pennsylvania,Northeastern,38,Pennsylvania,0.603620,947.479246,899.387875,79753.223857,70024.074750,...,0.401324,0.337089,0.248054,0.113487,0.487534,0.512466,48.091371,9729.149107,84.492160,27.147733


In [23]:
gender_demographics = pd.read_csv('Datasets/Discoveries/State_Gender_split_Degrees.csv')
state_lookup = state_lookup.merge(gender_demographics, how='outer',left_on='State', right_on='State')

In [24]:
gender_demographics.head(3)

,State,Female,Male,Ratio
0,Alabama,959478.0,811236.0,1.182736
1,Alaska,160674.0,131640.0,1.220556
2,Arizona,1536138.0,1448178.0,1.060738


In [25]:
Age_demographics = pd.read_csv('Datasets/Discoveries/State_Income_Age_Stacked_Degrees.csv')
state_lookup = state_lookup.merge(Age_demographics, how='outer',left_on='State', right_on='State')

In [26]:
Age_demographics.head(3)

,State,25 and older,25 to 39,40 to 64,65 and older,young_workers,old_workers
0,District of Columbia,301429.0,163151.0,98432.0,39846.0,0.541258,0.132190
1,North Dakota,153397.0,60715.0,64094.0,28588.0,0.395803,0.186366
2,Utah,664661.0,257758.0,289564.0,117339.0,0.387804,0.176540


In [27]:
top_10_demographics = pd.read_csv('Datasets/Discoveries/State_Top_10_Population_Mean_List.csv')
state_lookup = state_lookup.merge(top_10_demographics, how='outer',left_on='State', right_on='State')

In [28]:
top_10_demographics.head(3)

,State,Top_10_Mean_Pop
0,Alabama,284410.7000
1,Alaska,105006.8875
2,Arizona,888699.7375


In [29]:
degree_counts = pd.read_csv('Datasets/Discoveries/State_Degree_Counts.csv')
state_lookup = state_lookup.merge(degree_counts, how='outer',left_on='State', right_on='State')

In [30]:
degree_counts.head(3)

,State,Bachelor's Degree Holders,Science and Engineering,Science and Engineering Related Fields,Business,Education,"Arts, Humanities and Others",sci_eng,sci_eng_rel,business,education,art_hum_oth
0,District of Columbia,602858.0,292418.0,31224.0,78228.0,20502.0,180486.0,0.485053,0.051793,0.129762,0.034008,0.299384
1,Washington,3911264.0,1647784.0,333010.0,626424.0,376178.0,927868.0,0.421292,0.085141,0.160159,0.096178,0.237230
2,California,18856968.0,7859122.0,1562792.0,3442188.0,1160732.0,4832134.0,0.416775,0.082876,0.182542,0.061555,0.256252


In [31]:
saletime_demographics = pd.read_csv('Datasets/Discoveries/State_Cost_to_saletime_Comparison_Scores.csv')
saletime_demographics = saletime_demographics[saletime_demographics['Year'] == 2020]
saletime_demographics = saletime_demographics[['State', 'Days', 'Price', 'score']]
state_lookup = state_lookup.merge(saletime_demographics, how='outer',left_on='State', right_on='State')

In [32]:
tier_df = pd.read_csv('Final_Data/Tiered_State_Rental_Prices.csv')
tier_df.columns = ['State_Rental_Tier', 'StateAbbr', 'Mean_Rental_Value']
              
state_lookup = state_lookup.merge(tier_df, how='outer',left_on='StateAbbreviation', right_on='StateAbbr')

In [33]:
state_lookup = state_lookup[['StateAbbreviation', 'State', 'Region', 'State_Rental_Tier', 'Mean_Rental_Value','debt', 'rent_mean',
       'rent_median', 'family_mean', 'family_median', 'hc_mortgage_mean',
       'hc_mortgage_median', 'hc_mean', 'hc_median', 'hs_degree',
       'hs_degree_male', 'hs_degree_female', 'male_age_mean',
       'male_age_median', 'female_age_mean', 'female_age_median', 'hi_mean',
       'hi_median', 'pct_own', 'married', 'married_snp', 'separated',
       'divorced', 'rent_gt_10', 'rent_gt_15', 'rent_gt_20', 'rent_gt_25',
       'rent_gt_30', 'rent_gt_35', 'rent_gt_40', 'rent_gt_50', 'home_equity',
       'male_percent', 'female_percent', 'rent_mean_to_median',
       'family_mean_to_median', 'hc_mortgage_mean_to_median',
       'hc_mean_to_median', 'Female', 'Male', 'Ratio', '25 and older',
       '25 to 39', '40 to 64', '65 and older', 'young_workers', 'old_workers',
       'Top_10_Mean_Pop', 'Bachelor\'s Degree Holders',
       'Science and Engineering', 'Science and Engineering Related Fields',
       'Business', 'Education', 'Arts, Humanities and Others', 'sci_eng',
       'sci_eng_rel', 'business', 'education', 'art_hum_oth',
       'Days', 'Price', 'score']]

In [34]:
state_lookup = state_lookup.rename(columns = {'Female': 'Female_Degree_Holders', 'Male': 'Male_Degree_Holders', 'Ratio':'Gender_Degree_Ratio', 'sci_eng': 'sci_eng_pct',
       'sci_eng_rel': 'sci_eng_rel_pct', 'business': 'business_pct', 'education': 'education_pct', 'art_hum_oth': 'art_hum_ot_pct', 'score':'house_market_score'})

In [35]:
state_lookup.to_csv('Final_Data/Full_City_Lookup.csv', index=False)